In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.font_manager as fm
import koreanize_matplotlib

# 폰트 경로 확인 및 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # 설치된 폰트 경로
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

# 한글 깨짐 방지 - 마이너스 기호 처리
plt.rcParams['axes.unicode_minus'] = False

In [5]:
# 파일 불러오기
v5_path = "/home/eunyu/master_last_v5.csv"
all_path = "/home/eunyu/master_with_all.csv"

v5_df = pd.read_csv(v5_path)
all_df = pd.read_csv(all_path)

# 각 데이터프레임의 shape 확인
v5_df.shape, all_df.shape


((2507, 23), (4845, 19))

In [6]:
v5_df

,Unnamed: 0,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,...,votes_within_3d,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category,chosen_count
0,0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,...,77,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일'],342.0
1,1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,...,10,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계'],40.0
2,2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,...,69,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,['상상'],309.0
3,3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,...,259,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,['스타일'],256.0
4,4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,...,73,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,['스타일'],201.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,2502,1497413,335,13,2023-05-24,2023-07-28,65,2023-05-24,0.294118,M,...,152,35.0,bottom25,경상남도 거제시,483,H,1046,졸업 후 동창회에서 청첩장 돌릴 것 같은 친구는?,['인간관계'],199.0
2503,2503,1497699,15,2,2023-05-24,2023-05-25,1,2023-05-24,0.294118,M,...,15,2.0,bottom25,충청남도 천안시 서북구,491,H,336,제대로 놀 때 부르고 싶은 친구는?,['인간관계'],12.0
2504,2504,1498168,268,15,2023-05-24,2023-06-09,16,2023-05-24,0.294118,M,...,54,23.0,bottom25,경상남도 거제시,483,H,911,답장 속도가 가장 빠른 사람은?,['인간관계'],146.0
2505,2505,1498266,4,1,2023-05-24,2023-05-24,0,2023-05-24,0.294118,F,...,4,28.0,bottom25,울산광역시 울주군,550,H,914,예술감각이 가장 뛰어난 친구는?,['스타일'],34.0


# 상위 하위 25% 지표

In [11]:
# school_type 값 고유 확인
v5_df["school_type"].value_counts(dropna=False)



school_type
H    2000
M     507
Name: count, dtype: int64

- 고등학생: 2000명
- 중학생: 507명

In [ ]:
v5_df.columns

Index(['Unnamed: 0', 'user_id', 'total_votes', 'unique_days', 'first_vote',
       'last_vote', 'active_days', 'cohort_day', 'retention_day8', 'gender',
       'school_id', 'grade', 'group_id', 'votes_within_3d',
       'period_friend_count', 'retention_group', 'address', 'student_count',
       'school_type', 'firstquestion_id', 'first_question_text',
       'first_question_category', 'chosen_count'],
      dtype='object')

### 코호트 분석 학교 타입

In [10]:
# 중/고등학생 비율 다시 계산
cohort_summary = (
    v5_df[v5_df["retention_group"].isin(["top25", "bottom25"])]
    .groupby(["retention_group", "school_type"])["user_id"]
    .count()
    .unstack(fill_value=0)
)

# 비율 계산 (H: 고등학생, M: 중학생)
cohort_summary["중학생 비중"] = cohort_summary.get("M", 0) / cohort_summary.sum(axis=1)
cohort_summary["고등학생 비중"] = cohort_summary.get("H", 0) / cohort_summary.sum(axis=1)

# 결과 정리
cohort_summary_result = cohort_summary[["고등학생 비중", "중학생 비중"]] * 100
cohort_summary_result = cohort_summary_result.round(1).reset_index().rename(columns={"retention_group": "cohort"})

display(cohort_summary_result)

school_type,cohort,고등학생 비중,중학생 비중
0,bottom25,95.4,4.6
1,top25,64.4,35.6


→ 중학생 비중이 상위 리텐션 그룹에서 유의하게 높음

#### z-test

In [13]:
import sys
sys.path.append('/home/eunyu/.local/share/pipx/shared/lib/python3.12/site-packages')
from statsmodels.stats.proportion import proportions_ztest

# 중학생 수
m_top = v5_df[(v5_df["retention_group"] == "top25") & (v5_df["school_type"] == "M")].shape[0]
m_bottom = v5_df[(v5_df["retention_group"] == "bottom25") & (v5_df["school_type"] == "M")].shape[0]

# 전체 수
n_top = v5_df[v5_df["retention_group"] == "top25"].shape[0]
n_bottom = v5_df[v5_df["retention_group"] == "bottom25"].shape[0]

# z-test for two proportions
count = [m_top, m_bottom]
nobs = [n_top, n_bottom]

z_stat, p_val = proportions_ztest(count, nobs)

z_stat, p_val


(19.35067561685433, 2.0118760068034934e-83)

- Z-statistic: 19.35
- P-value: ≈ 2.01 × 10⁻⁸³ (매우 작음)



- 상위 25% 리텐션 그룹과 하위 25% 그룹 간의 중학생 비율 차이는 통계적으로 유의미
- P-value가 0.05보다 훨씬 작기 때문에, → "상위 리텐션 그룹에서 중학생 비중이 높다"는 현상은 우연이 아닐 가능성이 매우 큼을 의미
- 즉, 중학생은 리텐션이 좋은 유저군으로 간주할 수 있으며, 중학생을 타겟으로 한 전략 수립의 근거가 충분

In [14]:
# days_to_vote (활동 기간) = last_vote - first_vote + 1
v5_df["first_vote"] = pd.to_datetime(v5_df["first_vote"], errors="coerce")
v5_df["last_vote"] = pd.to_datetime(v5_df["last_vote"], errors="coerce")
v5_df["active_days"] = (v5_df["last_vote"] - v5_df["first_vote"]).dt.days + 1

# vote_acquisition_rate = chosen_count / active_days
v5_df["vote_acquisition_rate"] = v5_df["chosen_count"] / v5_df["active_days"]

# 중학생 + 리텐션 그룹 필터
middle_v5_df = v5_df[(v5_df["school_type"] == "M") & v5_df["retention_group"].isin(["top25", "bottom25"])]

# 그룹별 평균 비교
grouped_rate = (
    middle_v5_df.groupby("retention_group")["vote_acquisition_rate"]
    .agg(["mean", "count"])
    .rename(columns={"mean": "평균 투표 획득률", "count": "유저 수"})
    .reset_index()
)

# Welch t-test & Mann-Whitney U test
from scipy.stats import ttest_ind, mannwhitneyu

top = middle_v5_df[middle_v5_df["retention_group"] == "top25"]["vote_acquisition_rate"].dropna()
bottom = middle_v5_df[middle_v5_df["retention_group"] == "bottom25"]["vote_acquisition_rate"].dropna()

t_stat, t_pval = ttest_ind(top, bottom, equal_var=False)
u_stat, u_pval = mannwhitneyu(top, bottom, alternative='two-sided')

grouped_rate, (t_stat, t_pval), (u_stat, u_pval)


(  retention_group  평균 투표 획득률  유저 수
 0        bottom25   8.129761    57
 1           top25  17.573497   450,
 (4.125825724789996, 6.844504811367063e-05),
 (19799.5, 2.19184263659001e-11))

통계 검정 결과               
- Welch’s t-test → t = 4.13, p = 0.00007 
- Mann-Whitney U test → U = 19,799.5, p = 2.19e-11

- 상위 리텐션 집단은 하루 평균 17.6건의 투표를 받았으며,             
하위 집단은 8.1건에 불과해 2배 이상의 차이가 난다.           
- 두 검정 모두에서 p < 0.001이므로,→ 해당 차이는 우연이 아닌 통계적으로 유의한 차이       

- 많은 투표를 받은 중학생일수록 리텐션이 높다는 가설을 강하게 지지한다.

## A/B 테스트 설계

In [15]:
users_df = pd.read_parquet("gs://final_project_enuyu/data/final_project/votes/accounts_user.parquet")

In [17]:
users_df["created_at"] = pd.to_datetime(users_df["created_at"])
v5_df["first_vote"] = pd.to_datetime(v5_df["first_vote"])

users_df = users_df.rename(columns={"id": "user_id"})

#  가입일 병합
days_to_vote_df = v5_df[["user_id", "first_vote"]].merge(
    users_df[["user_id", "created_at"]],
    on="user_id",
    how="left"
)

# days_to_vote 계산
days_to_vote_df["days_to_vote"] = (days_to_vote_df["first_vote"] - days_to_vote_df["created_at"]).dt.days
days_to_vote_df["days_to_vote"] = days_to_vote_df["days_to_vote"].apply(lambda x: max(x, 0))

# 최종 마스터 병합
final_master_df = v5_df.merge(
    days_to_vote_df[["user_id", "days_to_vote"]],
    on="user_id",
    how="left"
)

In [19]:
v5_df.columns

Index(['Unnamed: 0', 'user_id', 'total_votes', 'unique_days', 'first_vote',
       'last_vote', 'active_days', 'cohort_day', 'retention_day8', 'gender',
       'school_id', 'grade', 'group_id', 'votes_within_3d',
       'period_friend_count', 'retention_group', 'address', 'student_count',
       'school_type', 'firstquestion_id', 'first_question_text',
       'first_question_category', 'chosen_count', 'days_to_vote',
       'vote_acquisition_rate'],
      dtype='object')

### Treatment 그룹 평균

In [21]:
# Treatment 그룹 정의: votes_within_3d > 0 (즉, 3일 내 투표 참여한 유저)
treatment_group = v5_df[v5_df["votes_within_3d"] > 0]

# Treatment 그룹 평균 계산
treatment_summary = {
    "days_to_vote_mean": treatment_group["days_to_vote"].mean(),
    "votes_within_3d_mean": treatment_group["votes_within_3d"].mean(),
    "retention_day8_mean": treatment_group["retention_day8"].mean(),
    "chosen_count_mean": treatment_group["chosen_count"].mean()
}

treatment_summary

{'days_to_vote_mean': 19.08336657359394,
 'votes_within_3d_mean': 108.47945751894694,
 'retention_day8_mean': 0.4579178300757878,
 'chosen_count_mean': 216.3320031923384}

In [23]:
# votes_within_3d == 0인 유저 수 확인
control_zero_count = (v5_df["votes_within_3d"] == 0).sum()
control_zero_count


0

###  Control 그룹과 직접 비교하여 통계적 유의성 분석

In [24]:
# days_to_vote ≤ 3일 vs > 3일 기준으로 그룹 분할
treatment_group = v5_df[v5_df["days_to_vote"] <= 3]
control_group = v5_df[v5_df["days_to_vote"] > 3]

# 리텐션 컬럼 결측치 제거
treatment_ret = treatment_group["retention_day8"].dropna()
control_ret = control_group["retention_day8"].dropna()

# t-test 수행
t_stat, p_val = ttest_ind(treatment_ret, control_ret, equal_var=False)

# 평균 리텐션 계산
treatment_mean = treatment_ret.mean()
control_mean = control_ret.mean()

{
    "t_stat": t_stat,
    "p_value": p_val,
    "treatment_mean": treatment_mean,
    "control_mean": control_mean,
    "treatment_n": len(treatment_ret),
    "control_n": len(control_ret)
}


{'t_stat': -12.073171489196145,
 'p_value': 9.441724875027655e-30,
 'treatment_mean': 0.3758342855102431,
 'control_mean': 0.47062168589063025,
 'treatment_n': 336,
 'control_n': 2171}